# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,28.15,95,62,2.35,AR,1723414493
1,1,whitehorse,60.7161,-135.0538,59.31,51,88,18.01,CA,1723414578
2,2,sabang,5.8933,95.3214,81.68,76,100,22.35,ID,1723414579
3,3,glendive,47.1053,-104.7125,65.82,77,0,3.44,US,1723414581
4,4,albany,42.6001,-73.9662,71.89,63,79,5.93,US,1723414507


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity', 
    color='City', 
    colorbar=True, 
    frame_width=800, 
    frame_height=600, 
    title='City Humidity Map',
    tiles = 'OSM'
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 10) & 
    (city_data_df["Max Temp"] < 100) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] > 20)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,28.15,95,62,2.35,AR,1723414493
4,4,albany,42.6001,-73.9662,71.89,63,79,5.93,US,1723414507
5,5,borovskoy,53.8000,64.1500,56.73,74,100,6.06,KZ,1723414583
7,7,hermanus,-34.4187,19.2345,54.54,73,85,8.77,ZA,1723414586
13,13,labuan,5.2767,115.2417,81.68,76,100,2.84,MY,1723414471


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8000,-68.3000,95,
4,albany,US,42.6001,-73.9662,63,
5,borovskoy,KZ,53.8000,64.1500,74,
7,hermanus,ZA,-34.4187,19.2345,73,
13,labuan,MY,5.2767,115.2417,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "limit": 1, 
    "apiKey": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
borovskoy - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
labuan - nearest hotel: Labuan Avenue Hotel
revda - nearest hotel: Металлург
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
waitangi - nearest hotel: Hotel Chathams
prado - nearest hotel: Pousada Recanto do Prado
saldanha - nearest hotel: Hoedjesbaai
richards bay - nearest hotel: SUN1 Richards Bay
st. john's - nearest hotel: Alt Hotel
grytviken - nearest hotel: No hotel found
badger - nearest hotel: Clarion Hotel & Suites
bredasdorp - nearest hotel: Victoria Hotel
ribeira grande - nearest hotel: Hospedaria JSF
okhotsk - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
talnakh - nearest hotel: Талнах
oldenburg in holstein - nearest hotel: Zur Eule
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
severo-yeniseys

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8000,-68.3000,95,Apart Hotel Aires del Beagle
4,albany,US,42.6001,-73.9662,63,No hotel found
5,borovskoy,KZ,53.8000,64.1500,74,No hotel found
7,hermanus,ZA,-34.4187,19.2345,73,Aloe guest house
13,labuan,MY,5.2767,115.2417,76,Labuan Avenue Hotel
...,...,...,...,...,...,...
568,yuzhno-kurilsk,RU,44.0306,145.8556,99,Айсберг
573,al muwayh,SA,22.4333,41.7583,16,فندق ومطعم الموية
574,carbonear,CA,47.7332,-53.2148,93,No hotel found
576,wukari,NG,7.8500,9.7833,93,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo=True, 
    size="Humidity", 
    color="City", 
    colorbar=True, 
    frame_width=800, 
    frame_height=600, 
    title="Hotel Weather Map",
    hover_cols=["Hotel Name", "Country"],
    tiles = "OSM"
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)